In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 57.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.0 MB/s

In [4]:
!pip install bertopic[visualization]

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-eymtx57q/kobert-tokenizer_e93a092197224e75aa56460572b695a1
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-eymtx57q/kobert-tokenizer_e93a092197224e75aa56460572b695a1
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=c4fea20a555956361cd36c21334fab297552b573ddd75f379a21f9be0b4f93bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-g2cw2gzg/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [6]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [7]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']), attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

torch.Size([1, 768])

In [8]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [46]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    # zip 파일 압축 해제
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
            data = json.load(f)
            valid = []
            for d in data['content'].split('\n'):
                res = []
                for word in d.split(' '):
                    if cond.match(word) != None:
                        res.append(word)
                d = ' '.join(res)
                if d and not d.replace(' ', '').isdecimal():
                    valid.append(d)
            result.append('\n'.join([data['title']] + valid))
    return result


In [52]:
import random
random.seed(42)
galleries = ['ktwiz','hanwhaeagles_new']#,'skwyverns_new1','ncdinos','samsunglions_new','doosanbears_new1','giants_new2','sh_new','lgtwins_new','tigers_new']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen

In [53]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split(' ') if word not in w.lower()])
stopwords = ['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']

In [54]:
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [55]:
bertopic_model = BERTopic(language='multilingual',
                          embedding_model=model,
                          # vectorizer_model=vectorizer,
                          nr_topics=50,
                          top_n_words=10,
                          calculate_probabilities=True)

In [56]:
topics, probs = bertopic_model.fit_transform(cocktail)

In [57]:
bertopic_model.visualize_topics()

In [58]:
bertopic_model.visualize_distribution(probs[0])

In [76]:
from sklearn.cluster import KMeans

n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_


In [77]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
from sklearn.metrics import confusion_matrix
print(len(real_labels))
print(confusion_matrix(real_labels, labels))


4757
[[2225   48]
 [2350  134]]


In [78]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

def calc_purity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    _, col_ind = linear_sum_assignment(cm, maximize=True)
    reordered_cm = cm[:, col_ind]
    purity = np.sum(np.max(reordered_cm, axis=0)) / np.sum(reordered_cm)
    return purity, reordered_cm

purity, reordered_cm = calc_purity(real_labels, labels)

print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

Purity:  0.5221778431784738
Reordered Confusion Matrix: 
 [[  48 2225]
 [ 134 2350]]


In [79]:
def show_cluster_examples(data, labels, n_clusters, examples_per_cluster=3):
    for cluster in range(n_clusters):
        print(f"Cluster {cluster + 1}:")
        cluster_indices = np.where(labels.astype(int) == cluster)[0]
        cluster_examples = [data[idx] for idx in cluster_indices]
        for example in cluster_examples[:examples_per_cluster]:
            print(example)
        print()

n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_
show_cluster_examples(cocktail, labels, n_clusters)


Cluster 1:
난 기아,ssg,한화가 싫음
개인적인 생각임.
뭔가 저 셋은 결이 비슷함.
자리있는데도 원정석 침범하는것 등등
표현 어려운 불쾌감이
저새끼 여기서 찍히고 레몬테라슨가 오렌지베란다로 가서
얘드라 나 찍혀소 오또케 이지랄하는거
올해 야구 첨보는데 전에는 이때 뭐뭐했음?

Cluster 2:
이따 올 콱붕이를 위한 겆붕열사 원투펀치 설명함
라이트 펀치
레프트 펀치
이 이후로 헬통년 글도 댓글도 안달린다
저 어그로 끄는 새끼 알고있는 겆붕이가 있어서 저격에 바로
박정현 언급
그 헬통 특) P드립 치면서 지랄하던데
알고보니 본인 개능지



In [80]:
len(dataset[1][0])

2484